# OpenMKM Input and Output
This notebook describes pmutt's functionality to write OpenMKM CTI and YAML files. We will use the NH3 formation mechanism as a case study.

## Topics Covered
- Read species *ab-initio* data, reactions, lateral interactions, phases, reactor operating conditions, and desired units from a spreadsheet
- Write the CTI file that can be read by OpenMKM
- Write a YAML file that can be read by OpenMKM

## Input Spreadsheet
All the data will be imported from the [`./inputs/NH3_Input_data.xlsx`](https://github.com/VlachosGroup/pMuTT/blob/master/docs/source/examples_jupyter/omkm_io/inputs/NH3_Input_Data.xlsx) file. There are several sheets:

1. `units` contains the units that types of quantities should be written
2. `refs` contains *ab-initio* and experimental data for a handful of gas species to calculate references (optional)
3. `species` contains *ab-initio* data for each specie
4. `beps` contains Bronsted-Evans-Polanyi relationships for reactions (optional)
5. `reactions` contains elementary steps 
6. `lateral_interactions` contains lateral interactions between species (optional)
7. `phases` contains phases for the species
8. `reactor` contains reactor operating conditions and solver tolerances

The ``refs``, ``beps`` and ``lateral_interactions`` sheets can be deleted and the code written below should still work.

First, we change the working directory to the location of the Jupyter notebook.

In [1]:
import os
from pathlib import Path

# Find the location of Jupyter notebook
# Note that normally Python scripts have a __file__ variable but Jupyter notebook doesn't.
# Using pathlib can overcome this limiation
try:
    notebook_path = os.path.dirname(__file__)
except NameError:
    notebook_path = Path().resolve()
    
os.chdir(notebook_path)
input_path = './inputs/NH3_Input_Data.xlsx'

And we define a helper function to print data from the Excel spreadsheet easily.

In [2]:
import pandas as pd
from IPython.display import display

def disp_data(io, sheet_name):
    try:
        data = pd.read_excel(io=io, sheet_name=sheet_name, skiprows=[1])
    except:
        print('The {} sheet could not be found in {}.'.format(sheet_name, io))
    else:
        data = data.fillna(' ')
        display(data)

Below, we show the contents of the Excel sheets.

**Units**

In [3]:
disp_data(io=input_path, sheet_name='units')

,length,quantity,act_energy,mass,energy,pressure
0,cm,mol,kcal/mol,g,kcal,atm


**References**

In [4]:
disp_data(io=input_path, sheet_name='refs')

,name,elements.N,elements.H,elements.Ru,T_ref,HoRT_ref,potentialenergy,symmetrynumber,statmech_model,atoms,vib_wavenumber,vib_wavenumber.1,vib_wavenumber.2,vib_wavenumber.3
0,N2,2,0,0,298.15,0.000000,-16.63,2,IdealGas,./N2/CONTCAR,2744,,,
1,NH3,1,3,0,298.15,-18.380253,-19.54,3,IdealGas,./NH3/CONTCAR,3534,3464,1765,1139
2,H2,0,2,0,298.15,0.000000,-6.77,2,IdealGas,./H2/CONTCAR,4342,,,
3,Ru,0,0,1,298.15,0.000000,,,Placeholder,,,,,


**Species**

In this mechanism, we have species existing on terraces and steps. We also define the transition states of species. 

Note that later we will use BEPs to calculate barriers for most steps. Hence, these transition state species will actually be ignored. You should use either transition state species or BEP relationships to calculate barriers.

In [5]:
disp_data(io=input_path, sheet_name='species')

,name,elements.N,elements.H,elements.Ru,phase,n_sites,T_low,T_high,statmech_model,symmetrynumber,...,vib_wavenumber.2,vib_wavenumber.3,vib_wavenumber.4,vib_wavenumber.5,vib_wavenumber.6,vib_wavenumber.7,vib_wavenumber.8,vib_wavenumber.9,vib_wavenumber.10,vib_wavenumber.11
0,N2,2,,,gas,,300,1000,IdealGas,2,...,,,,,,,,,,
1,NH3,1,3,,gas,,300,1000,IdealGas,3,...,1765,1139,,,,,,,,
2,H2,,2,,gas,,300,1000,IdealGas,2,...,,,,,,,,,,
3,N2(T),2,,,terrace,1,300,1000,Harmonic,,...,347.343,335.674,62.076,32.1794,,,,,,
4,N(T),1,,,terrace,1,300,1000,Harmonic,,...,504.323,475.805,459.081,410.018,,,,,,
5,H(T),,1,,terrace,1,300,1000,Harmonic,,...,616.29,,,,,,,,,
6,NH3(T),1,3,,terrace,1,300,1000,Harmonic,,...,3364.52,1583.52,1582.07,1124.22,570.212,567.221,333.09,122.859,83.8286,70.6251
7,NH2(T),1,2,,terrace,1,300,1000,Harmonic,,...,1503.02,698.869,625.596,615.94,475.13,298.12,153.25,,,
8,NH(T),1,1,,terrace,1,300,1000,Harmonic,,...,710.581,528.526,415.196,410.131,,,,,,
9,TS1_NH3(T),1,3,,,1,300,1000,Harmonic,,...,1723.85,1487.95,959.151,888.946,594.089,428.431,227.032,206.047,142.136,


**BEPs**

In [6]:
disp_data(io=input_path, sheet_name='beps')

,name,slope,intercept,direction,notes
0,N-H,0.29,23.23,cleavage,Values taken from https://github.com/VlachosGr...
1,NH-H,0.52,19.78,cleavage,Values taken from https://github.com/VlachosGr...
2,NH2-H,0.71,23.69,cleavage,Values taken from https://github.com/VlachosGr...


**Reactions**

Note that reactions with two '=' signs indicate it has a transition state or BEP relationship.

In [7]:
disp_data(io=input_path, sheet_name='reactions')

,reaction_str,is_adsorption,direction,beta,A,Ea,notes
0,H2 + 2RU(T) = 2H(T) + 2RU(B),True,,0,,,Adsorption reactions typically use beta = 0
1,N2 + RU(T) = N2(T) + RU(B),True,,0,,,
2,NH3 + RU(T) = NH3(T) + RU(B),True,,0,,,
3,NH3(T) + RU(T) = NH2(T) + H(T) + RU(B),False,,1,9.6e+17,14.2,Pre-exponential and activation energy set manu...
4,NH2(T) + RU(T) = NH-H = NH(T) + H(T) + RU(B),False,cleavage,1,,,
5,NH(T) + RU(T) = N-H = N(T) + H(T) + RU(B),False,cleavage,1,,,
6,2N(T) + RU(B) = TS4_N2(T) = N2(T) + RU(T),False,,1,,,
7,H2 + 2RU(S) = 2H(S) + 2RU(B),True,,0,,,
8,N2 + RU(S) = N2(S) + RU(B),True,,0,,,
9,NH3 + RU(S) = NH3(S) + RU(B),True,,0,,,


**Lateral Interactions**

Currently we use piece-wise linear equations for lateral interactions. Here, we only define one interval between 0 - 1 ML but additional ``list.intervals`` and ``list.slopes`` columns can be added for more complicated behavior.

In [8]:
disp_data(io=input_path, sheet_name='lateral_interactions')

,name_i,name_j,list.intervals,list.slopes,list.intervals.1,notes
0,N(T),N(T),0,-52.6,1,
1,N(T),H(T),0,-17.7,1,
2,H(T),N(T),0,-17.7,1,
3,H(T),H(T),0,-3.0,1,
4,NH2(T),N(T),0,-20.7,1,
5,N(S),N(S),0,-52.6,1,
6,N(S),H(S),0,-17.7,1,
7,H(S),N(S),0,-17.7,1,
8,H(S),H(S),0,-3.0,1,
9,NH2(S),N(S),0,-20.7,1,


**Phases**

As previously stated, there are two surface sites: terrace and step. There are also the gas and bulk phases defined.

In [9]:
disp_data(io=input_path, sheet_name='phases')

,name,phase_type,density,site_density,list.phases,list.phases.1,dict.initial_state.RU(T),dict.initial_state.RU(S),dict.initial_state.NH3,note
0,gas,IdealGas,,,,,,,1,
1,bulk,StoichSolid,12.4,,,,,,,Ru Metal
2,terrace,InteractingInterface,,2.1671e-09,gas,bulk,1,,,Ru(0001)
3,step,InteractingInterface,,4.4385e-10,gas,bulk,,1,,Ru(0001) with atoms deleted


**Reactor**

The reactor sheet contains options for the YAML file.

In [10]:
disp_data(io=input_path, sheet_name='reactor')

,reactor_type,mode,V,T,P,cat_abyv,end_time,flow_rate,transient,stepping,init_step,atol,rtol,output_format
0,cstr,isothermal,1,900,1,1500,50,1,True,logarithmic,1.000000e-15,1.000000e-15,1.000000e-10,csv


## Reading data

Throughout this exercise, we will use [``pmutt.io.read_excel``](https://vlachosgroup.github.io/pMuTT/api/io/excel/pmutt.io.excel.read_excel.html#pmutt-io-excel-read-excel) to extract the data from the Excel spreadsheet.

### Designate Units
First, we will designate the units to write the CTI and YAML file.

In [11]:
from pmutt.io.excel import read_excel
from pmutt.omkm.units import Units

units_data = read_excel(io=input_path, sheet_name='units')[0]
units = Units(**units_data)

### Reading References (optional)
Second, we will open the input spreadsheet and read the `refs` sheet.

In [12]:
from pmutt.empirical.references import Reference, References

try:
    refs_data = read_excel(io=input_path, sheet_name='refs')
except:
    # If references are not used, skip this section
    print('The "refs" sheet could not be found in {}. Skiping references'.format(input_path))
    refs = None
else:
    refs = [Reference(**ref_data) for ref_data in refs_data]
    refs = References(references=refs)

### Reading Species

Third, we will use the ``refs`` defined before and the ``species`` sheet to convert statistical mechanical data to [``NASA``](https://vlachosgroup.github.io/pMuTT/api/empirical/nasa/pmutt.empirical.nasa.Nasa.html#pmutt.empirical.nasa.Nasa) objects.

In [13]:
from pmutt.empirical.nasa import Nasa

# Read the species' data
species_data = read_excel(io=input_path, sheet_name='species')

# Create NASA polynomials from the species
species = [Nasa.from_model(references=refs, **ind_species_data) \
           for ind_species_data in species_data]

### Adding species from other empirical sources (optional)

Note that OpenMKM also supports [``Shomate``](https://vlachosgroup.github.io/pMuTT/api/empirical/shomate/pmutt.empirical.shomate.Shomate.html#pmutt.empirical.shomate.Shomate) and [``NASA9``](https://vlachosgroup.github.io/pMuTT/api/empirical/nasa/pmutt.empirical.nasa.Nasa9.html) objects. Below, we define a single ``Shomate`` species.

In [14]:
import numpy as np
from pmutt.empirical.shomate import Shomate

Ar = Shomate(name='Ar', elements={'Ar': 1}, phase='gas', T_low=298., T_high=6000.,
             a=np.array([20.78600, 2.825911e-7, -1.464191e-7, 1.092131e-8, -3.661371e-8, -6.19735, 179.999, 0.]))

species.append(Ar)

### Reading BEP (optional)

Next, we read the BEP relationships to include.

In [15]:
from pmutt.omkm.reaction import BEP

try:
    beps_data = read_excel(io=input_path, sheet_name='beps')
except:
    print('The "beps" sheet could not be found in {}. Skiping BEPs'.format(input_path))
    beps = None
    species_with_beps = species.copy()
else:
    beps = [BEP(**bep_data) for bep_data in beps_data]
    species_with_beps = species + beps

### Read reactions

Then, we read the reactions to include.

In [16]:
from pmutt import pmutt_list_to_dict
from pmutt.omkm.reaction import SurfaceReaction

# Convert species to dictionary for easier reaction assignment
species_with_beps_dict = pmutt_list_to_dict(species_with_beps)

reactions_data = read_excel(io=input_path, sheet_name='reactions')
reactions = [SurfaceReaction.from_string(species=species_with_beps_dict, **reaction_data) \
             for reaction_data in reactions_data]

### Read lateral interactions (optional)

After, we read lateral interactions to include.

In [17]:
from pmutt.mixture.cov import PiecewiseCovEffect

try:
    interactions_data = read_excel(io=input_path, sheet_name='lateral_interactions')
except:
    # If no lateral interactions exist, skip this section
    print('The "lateral_interactions" sheet could not be found in {}. Skiping lateral interactions'.format(input_path))
    interactions = None
else:
    interactions = [PiecewiseCovEffect(**interaction_data) for interaction_data in interactions_data]

### Reading Phases

Finally, we read the phases data from Excel and organize it for use in OpenMKM.

In [18]:
from pmutt.io.omkm import organize_phases

# Read data from Excel sheet about phases
phases_data = read_excel(io=input_path, sheet_name='phases')
phases = organize_phases(phases_data, species=species, reactions=reactions, interactions=interactions)

## Write Reactor YAML File

The YAML file specifying the reactor configuration can be written using the [``write_yaml``](https://vlachosgroup.github.io/pMuTT/api/kinetic_models/omkm/pmutt.io.omkm.write_yaml.html) function. Note that if:
- ``units`` is not specified, float values are assumed to be in SI units
- ``units`` is specified, float values are consistent with ``unit``'s attributes
- you would like a quantity to have particular units, pass the value as a string with the units  (e.g. "10 cm3/s").

In [19]:
from pmutt.io.omkm import write_yaml

Path('./outputs').mkdir(exist_ok=True)
yaml_path = './outputs/reactor.yaml'
reactor_data = read_excel(io=input_path, sheet_name='reactor')[0]
write_yaml(filename=yaml_path, phases=phases, units=units, **reactor_data)

If you would prefer to return the file as a string instead of writing it, omit the ``filename``.

In [20]:
print(write_yaml(phases=phases, units=units, **reactor_data))

# File generated by pMuTT (v 1.2.20) on 2020-05-17 14:00:44.114932
# See documentation for OpenMKM YAML file here:
# https://vlachosgroup.github.io/openmkm/input
inlet_gas:
    flow_rate: "1 cm3/s"
phases:
    bulk:
        name: bulk
    gas:
        initial_state: "NH3:1.0"
        name: gas
    surfaces:
    -   initial_state: "RU(T):1.0"
        name: terrace
    -   initial_state: "RU(S):1.0"
        name: step
reactor:
    cat_abyv: "1500 /cm"
    mode: "isothermal"
    pressure: "1 atm"
    temperature: 900
    type: "cstr"
    volume: "1 cm3"
simulation:
    end_time: "50 s"
    init_step: 1.0e-15
    output_format: "csv"
    solver:
        atol: 1.0e-15
        rtol: 1.0e-10
    stepping: "logarithmic"
    transient: true



## Write Thermo/Kinetic YAML File

As of OpenMKM version 0.6.0 onwards, the thermodynamic and kinetic parameters can be written as a YAML file. We recommend using this format over the older CTI format. To generate the Thermo/Kinetic YAML file using pMuTT, use the [``write_thermo_yaml``](https://vlachosgroup.github.io/pMuTT/api/kinetic_models/omkm/pmutt.io.omkm.write_thermo_yaml.html) function

In [21]:
from pmutt.io.omkm import write_thermo_yaml

write_thermo_yaml(phases=phases,
                  species=species,
                  reactions=reactions,
                  lateral_interactions=interactions,
                  units=units,
                  filename='./outputs/thermo.yaml')

Like before, omitting the ``filename`` parameter returns a string

In [22]:
print(write_thermo_yaml(phases=phases,
                        species=species,
                        reactions=reactions,
                        lateral_interactions=interactions,
                        units=units))

# File generated by pMuTT (v 1.2.20) on 2020-05-17 14:00:44.215241
# See documentation for OpenMKM YAML file here:
# https://vlachosgroup.github.io/openmkm/input

#-------------------------------------------------------------------------------
# UNITS
#-------------------------------------------------------------------------------
units: {mass: g, length: cm, time: s, quantity: mol, energy: kcal, activation-energy: kcal/mol,
  pressure: atm}


#-------------------------------------------------------------------------------
# PHASES
#-------------------------------------------------------------------------------
phases:

- name: gas
  elements: [H, Ar, N]
  species: [N2, NH3, H2, Ar]
  thermo: ideal-gas
  kinetics: gas
  reactions: none

- name: bulk
  elements: [Ru]
  species: [RU(B)]
  thermo: fixed-stoichiometry

- name: terrace
  elements: [Ru, H, N]
  species: [N2(T), N(T), H(T), NH3(T), NH2(T), NH(T), RU(T)]
  kinetics: surface
  site_density: "2.1671e-09 mol/cm2"
  thermo: surfac

## Write CTI File

The CTI file species the thermodynamics and kinetics of the system. It can be written using [``write_cti``](https://vlachosgroup.github.io/pMuTT/api/kinetic_models/omkm/pmutt.io.omkm.write_cti.html#pmutt.io.omkm.write_cti). Note that we take the reactor operating conditions previously read for the YAML file to calculate thermodynamic and kinetic parameters.

In [23]:
from pmutt.io.omkm import write_cti

cti_path = './outputs/thermo.cti'
use_motz_wise = True
T = reactor_data['T']

write_cti(reactions=reactions, species=species, phases=phases, units=units,
         lateral_interactions=interactions, filename=cti_path,
         use_motz_wise=use_motz_wise, T=T, P=1.)

Like before, omitting the ``filename`` parameter returns a string.

In [24]:
print(write_cti(reactions=reactions, species=species, phases=phases, units=units,
               lateral_interactions=interactions, use_motz_wise=use_motz_wise))

# File generated by pMuTT (v 1.2.20) on 2020-05-17 14:00:44.353263
# See documentation for OpenMKM CTI file here:
# https://vlachosgroup.github.io/openmkm/input

#-------------------------------------------------------------------------------
# UNITS
#-------------------------------------------------------------------------------
units(length="cm", time="s", quantity="mol", energy="kcal",
      act_energy="kcal/mol", pressure="atm", mass="g")

#-------------------------------------------------------------------------------
# PHASES
#-------------------------------------------------------------------------------
ideal_gas(name="gas",
          elements="H Ar N",
          species="N2 NH3 H2 Ar",
          reactions=[])

stoichiometric_solid(name="bulk",
                     elements="Ru",
                     species="RU(B)",
                     density=12.4,
                     note="Ru Metal")

interacting_interface(name="terrace",
                      elements="Ru H N",
          